# Entendimento dos dados

Este notebook tem a finalidade de proporcionar um maior entendimento a cerca dos dados disponibilizados.

* Para todas as bases serão realizadas as seguintes análises/operações:
    - Leitura do dataset
    - Informações a respeito das colunas
    - Quantidade de registros
    - Quantidade de valores faltantes
    - Separação de variáveis 


**Sumário**
1. Importação das bibliotecas
2. base_cadastral.csv
3. base_info.csv
4. base_pagamentos_desenvolvimento.csv

---

### 1. Importação das bibliotecas

**Bibliotecas utilizadas:**
- pandas: Manipulação de tabelas
- numpy: Manipulação numérica
- warnings: Evitar avisos indesejados

In [27]:
import pandas as pd
import numpy as np

from warnings import filterwarnings

from src.eda_utils import DataUtils

# Ignorar avisos
filterwarnings('ignore')

# Configs pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


---

### 2. base_cadastral.csv

**2.1 Leitura do dataset**

In [28]:
base_cadastral = DataUtils.read_data('base_cadastral.csv', info=True)

..\data\unprocessed\base_cadastral.csv
----------------------------------------------------------------------------------------------------

Shape

linhas: 1315 | colunas: 8

----------------------------------------------------------------------------------------------------

INFO

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_CLIENTE           1315 non-null   int64 
 1   DATA_CADASTRO        1315 non-null   object
 2   DDD                  1078 non-null   object
 3   FLAG_PF              66 non-null     object
 4   SEGMENTO_INDUSTRIAL  1232 non-null   object
 5   DOMINIO_EMAIL        1285 non-null   object
 6   PORTE                1274 non-null   object
 7   CEP_2_DIG            1315 non-null   object
dtypes: int64(1), object(7)
memory usage: 82.3+ KB
None

----------------------------------------------------------------

**Sobre a base**
- A base cadastral possui informações a respeito do cadastro realizado dos clientes. Dessa forma, nela estão contidas informações essenciais sobre o cliente.
- Ao todo, a base possui 1315 registros e 8 colunas


**Sobre as colunas**
- ID_CLIENTE: 
    - Id único utilizado para cadastrar os clientes; 
    - Sem valores nulos;
    - 1315 clientes identificados.
- DATA CADASTRO:
    - Data de cadastro do cliente;
    - Sem valores nulos;
    - 777 datas identificadas.
- DDD:
    - Identificador de Discagem Direta à Distância;
    - 237 valores nulos encontrados, representam 14,45% dos registros presentes;
    - 79 valores identificados, alguns estão com erros (provalvemente de inserção na tabela, pois começam com "(");
    - Presença de DDDs inválidos, como o 52 e o 00.
- FLAG_PF:
    - Indica se o cliente é pessoa física ou não;
    - Valores nulos identificados, entretanto, essa ausência significa que o cliente **não é** uma pessoa física;
    - 1 Valor único identificado ('X' em caso afirmativo do cliente ser pessoa física).
- SEGMENTO_INDUSTRIAL:
    -Identifica qual segmento o cliente atua;
    - 83 valores nulos identificados, representando 5,06% dos registros;
    - 3 valores únicos identificados.
- DOMINIO_EMAIL:
    - Identifica qual o domínio do e-mail utilizado para cadastro do cliente;
    - 30 valores nulos identificados, representando 1,83% dos registros;
    - 6 valores únicos identificados.
- PORTE:
    - Identifica o porte do cliente;
    - 41 valores nulos encontrados, representando cerca de 2,50% dos registros;
    - 3 valores únicos identificados.
- CEP_2_DIG:
    - Indica os dois primeiros dígitos do CEP do cliente;
    - Valores nulos identificados, porém não parametrizados para serem capturados via método '.isna()';
    - 90 valores únicos identificados

**Próximos passos**
- Para garantir a integridade da modelagem, bem como da ánalise, as seguintes correções/alterações serão feitas nas colunas:
    1. Mapeamento da coluna FLAG_PF, transformando 'NaN's em 0 e 'X' em 1
    2. Alteração da coluna DATA_CADASTRO para datetime pandas
    3. Entedimento sobre DDDs incongruentes ou com erros
    4. Correção de NaNs na coluna CEP_2_DIG


In [29]:
base_cadastral['FLAG_PF'].fillna(0, inplace=True)
base_cadastral.loc[base_cadastral['FLAG_PF'] == 'X', 'FLAG_PF'] = 1

base_cadastral['FLAG_PF'].unique()

array([0, 1], dtype=object)

In [30]:
base_cadastral['DATA_CADASTRO'] = pd.to_datetime(base_cadastral['DATA_CADASTRO'])

In [31]:
base_cadastral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315 entries, 0 to 1314
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_CLIENTE           1315 non-null   int64         
 1   DATA_CADASTRO        1315 non-null   datetime64[ns]
 2   DDD                  1078 non-null   object        
 3   FLAG_PF              1315 non-null   object        
 4   SEGMENTO_INDUSTRIAL  1232 non-null   object        
 5   DOMINIO_EMAIL        1285 non-null   object        
 6   PORTE                1274 non-null   object        
 7   CEP_2_DIG            1315 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 82.3+ KB


In [32]:
not_available_ddds = ['20', '23', '25', '26', '29', '30', '36', '39', '40', '50', '56', '57', '58', '59', '60', '70', '72', '76', '78', '80', '90', '00']

# Filtro
inconsistences_registers_bcs = base_cadastral.loc[(base_cadastral['DDD'].isin(not_available_ddds)) | (base_cadastral['DDD'].str.startswith('(')), :]

print('Quantidade de registros incongruentes presentes:')
print(inconsistences_registers_bcs.shape[0])
print()
inconsistences_registers_bcs

Quantidade de registros incongruentes presentes:
99



,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
55,2025898153078562640,2007-06-17,(3,0,Serviços,GMAIL,PEQUENO,36
70,3218792504195936313,2000-08-15,(2,0,Serviços,HOTMAIL,MEDIO,21
124,6759864637454708017,2000-08-15,(6,0,Serviços,GMAIL,GRANDE,68
156,1210783884008131648,2000-08-15,(1,0,Serviços,YAHOO,MEDIO,13
197,7457803471605010846,2000-08-15,56,0,Serviços,GMAIL,GRANDE,93
213,5791518044186095039,2000-08-15,(9,0,Serviços,HOTMAIL,PEQUENO,68
254,1088521227159442264,2000-08-15,(4,0,Serviços,YAHOO,PEQUENO,89
279,1553628056929342565,2009-09-29,(8,0,Comércio,YAHOO,MEDIO,59
362,6987942592445950913,2000-08-15,(1,0,Serviços,HOTMAIL,MEDIO,20
427,6858769421191999702,2000-08-15,00,0,Serviços,YAHOO,GRANDE,13


In [33]:
mask_ddd_bc = base_cadastral['DDD'].isin(not_available_ddds) | base_cadastral['DDD'].str.startswith('(')
base_cadastral.loc[mask_ddd_bc, 'DDD'] = 'INVÁLIDO' 

base_cadastral.loc[base_cadastral['DDD'] == mask_ddd_bc]

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG


Optarei por não excluir registros por hora, essa decisão será tomada durante a fase de EDA do processo, dessa forma, devemos marcar estas incongruências com uma flag. Portanto, estes registros terão seus DDDs alterados para 'INVÁLIDO'.

In [34]:
base_cadastral.loc[base_cadastral['CEP_2_DIG'] == 'na', :]

,ID_CLIENTE,DATA_CADASTRO,DDD,FLAG_PF,SEGMENTO_INDUSTRIAL,DOMINIO_EMAIL,PORTE,CEP_2_DIG
934,3903290047715748290,2019-10-07,NaN,0,Indústria,HOTMAIL,PEQUENO,na


In [35]:
base_cadastral.loc[base_cadastral['CEP_2_DIG'] == 'na', 'CEP_2_DIG'] = np.nan

In [36]:
base_cadastral['DDD'].str.startswith('(').sum()

0

---

### 3. base_info.csv

**3.1 Leitura do dataset**

In [37]:
base_info = DataUtils.read_data('base_info.csv', info=True)

..\data\unprocessed\base_info.csv
----------------------------------------------------------------------------------------------------

Shape

linhas: 24401 | colunas: 4

----------------------------------------------------------------------------------------------------

INFO

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24401 entries, 0 to 24400
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID_CLIENTE          24401 non-null  int64  
 1   SAFRA_REF           24401 non-null  object 
 2   RENDA_MES_ANTERIOR  23684 non-null  float64
 3   NO_FUNCIONARIOS     23149 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 762.7+ KB
None

----------------------------------------------------------------------------------------------------

Valores únicos


Coluna: ID_CLIENTE
1336

Coluna: SAFRA_REF
40
['2018-09' '2018-10' '2018-11' '2018-12' '2019-01' '2019-02' '2019-03'
 '2019-04' '2

**Sobre a base**
- A base info tem o intuito de compor informações adicionais a respeito dos clientes. Ela é atualizada mensalmente, e, cada cliente só aparece uma vez por mês (SAFRA_REF).
- A base possui 24401 registros e 4 colunas


**Sobre as colunas**
- ID_CLIENTE:
    - Mesma função da 'base_cadastral';
    - Sem valores nulos identificados;
    - 1316 valores únicos identificados, o que sugere uma incongruência em relação a 'base_cadastral' que possui 1315 valores únicos.
- SAFRA_REF:
    - Mesma função da 'base_cadastral';
    - Sem valores nulos identificados;
    - 40 valores únicos identificados. Como SAFRA_REF determina os meses em que estão sendo observados os registros, podemos assumir que existe um intervalo de 40 meses entre os dados, ou, 3 anos e 4 meses.
- RENDA_MES_ANTERIOR:
    - Identifica a renda ou o faturamento declarado pelo cliente no mês anterior;
    - 717 valores nulos identificados, representando 2,94% de todos os registros;
    - 23196 valores únicos identificados.
- NO_FUNCIONARIOS:
    - Número de funcionários declarado pelo cliente no mês anterior;
    - 1252 valores nulos identificados, representando 5,13% de todos os registros;
    - 128 valores únicos identificados.

**Próximos passos**
- Para garantir a integridade da modelagem, bem como da ánalise, as seguintes correções/alterações serão feitas nas colunas:
    1. Mudança de tipo de SAFRA_REF para datetime pandas


**3.2 Informações sobre as colunas**

In [38]:
base_info['SAFRA_REF'] = pd.to_datetime(base_info['SAFRA_REF'])

In [39]:
base_info[['RENDA_MES_ANTERIOR', 'NO_FUNCIONARIOS']].describe().T

,count,mean,std,min,25%,50%,75%,max
RENDA_MES_ANTERIOR,23684.0,288751.395035,211594.788834,105.0,133866.25,240998.5,392501.75,1682759.0
NO_FUNCIONARIOS,23149.0,117.799646,21.464574,0.0,106.00,118.0,131.00,198.0


Alguns insights:
1. RENDA_MES_ANTERIOR -> provavelmente apresentará uma distribuição assimétrica, com cauda alongada para a direita. Isso é denotado tanto pelos valores observados no Q1 e Q3, quanto no desvio padrão
2. NO_FUNCIONARIOS -> Apresentará uma distribuição normal, em torno de 118 funcionários como média.

Vamos verificar qual ID está presente aqui e não está em 'base_cadastral'

In [40]:
missing_id = base_info.loc[~(base_info['ID_CLIENTE'].isin(base_cadastral['ID_CLIENTE']))]

missing_clients = missing_id['ID_CLIENTE'].unique()
len(missing_clients), missing_clients[:10]

(21,
 array([5562543665139612149, 1166653268490470189, 8750634267020436314,
        9016049685460415544, 4228701400180066884, 1040994770557588198,
        5706551050445880395,  493667100055938118, 7662578945735335954,
          51762959302415533]))

Optarei por remover estes clientes, visto que 'base_cadastral' é a referência de cadastro de clientes, portanto, ambas as bases, devem ser correspondentes.

In [41]:
base_info = base_info[~base_info['ID_CLIENTE'].isin(missing_clients)]


base_info.loc[base_info['ID_CLIENTE'].isin(missing_clients)]

,ID_CLIENTE,SAFRA_REF,RENDA_MES_ANTERIOR,NO_FUNCIONARIOS


In [42]:
base_info.shape

(24371, 4)

---

### 4. base_pagamentos_desenvolvimento.csv

**4.1 Leitura do dataset**

In [43]:
base_pag = DataUtils.read_data('base_pagamentos_desenvolvimento.csv', info=True)

..\data\unprocessed\base_pagamentos_desenvolvimento.csv
----------------------------------------------------------------------------------------------------

Shape

linhas: 77414 | colunas: 7

----------------------------------------------------------------------------------------------------

INFO

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77414 entries, 0 to 77413
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_CLIENTE              77414 non-null  int64  
 1   SAFRA_REF               77414 non-null  object 
 2   DATA_EMISSAO_DOCUMENTO  77414 non-null  object 
 3   DATA_PAGAMENTO          77414 non-null  object 
 4   DATA_VENCIMENTO         77414 non-null  object 
 5   VALOR_A_PAGAR           76244 non-null  float64
 6   TAXA                    77414 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 4.1+ MB
None

----------------------------------------------

**Sobre a base**
- A base pagamentos mostra como são os pagamentos dos clientes presentes;
- A base possui 77414 registros e 7 colunas.

**Sobre as colunas**
- ID_CLIENTE:
    - Mesma função da 'base_cadastral';
    - Sem valores nulos identificados;
    - 1248 valores únicos identificados, o que sugere uma incongruência em relação a 'base_cadastral' que possui 1315 valores únicos.
- SAFRA_REF:
    - Mesma função da 'base_cadastral';
    - Sem valores nulos identificados;
    - 35 valores únicos identificados. Como SAFRA_REF determina os meses em que estão sendo observados os registros, podemos assumir que existe um intervalo de 35 meses entre os dados, ou, 2 anos e 11 meses.
- DATA_EMISSAO_DOCUMENTO:
    - Identifica o dia em que foi realizado o pagamento da cobrança;
    - Sem valores nulos identificados;
    - 921 valores únicos identificados.
- DATA_VENCIMENTO:
    - Identifica a data limite para realizar o pagamento da cobrança;
    - Sem valores nulos identificados;
    - 955 valores únicos identificados.
- VALOR_A_PAGAR:
    - Identifica o valor a ser pago na cobrança;
    - 1170 valores nulos identificados, representando 1,51% de todos os registros;
    - 67588 valores únicos identificados.
- TAXA:
    - Identifica a taxa de juros cobrada no empréstimo;
    - Sem valores nulos identificados;
    - 5 valores únicos identificados (forte evidência de ser uma variável categórica, pois "segmenta" clientes)


**Próximos passos**
- Para garantir a integridade da modelagem, bem como da ánalise, as seguintes correções/alterações serão feitas nas colunas:
    1. Mudança de tipo de SAFRA_REF para datetime pandas
    2. Check se todos os ID_CLIENTE presentes batem com os da 'base_cadastral'


In [44]:
base_pag.describe()

,ID_CLIENTE,VALOR_A_PAGAR,TAXA
count,7.741400e+04,7.624400e+04,77414.000000
mean,4.662270e+18,4.659078e+04,6.789623
std,2.665719e+18,4.643393e+04,1.798225
min,8.784237e+15,1.000000e-01,4.990000
25%,2.369365e+18,1.876536e+04,5.990000
50%,4.817817e+18,3.475869e+04,5.990000
75%,6.969349e+18,6.093384e+04,6.990000
max,9.206031e+18,4.400000e+06,11.990000


In [45]:
datetime_cols = ['SAFRA_REF', 'DATA_EMISSAO_DOCUMENTO', 'DATA_PAGAMENTO', 'DATA_VENCIMENTO']

base_pag[datetime_cols] = base_pag[datetime_cols].apply(pd.to_datetime)

In [46]:
missing_id = base_pag.loc[~(base_pag['ID_CLIENTE'].isin(base_cadastral['ID_CLIENTE']))]

missing_clients = missing_id['ID_CLIENTE'].unique()
len(missing_clients), missing_clients[:10]

(0, array([], dtype=int64))

Isso denota que a diferença de registros apenas mostra que existem clientes cadastrados que não possuem movimentações registradas no período de tempo observado. Além disso, optarei por hora, não excluir ou inserir valores em registros que possuem valores faltantes na coluna VALOR_A_PAGAR com o intuito de evitar data leakeage. Essa inserção/remoção ocorrerá durante a EDA.

---


**Processamento das bases**


Para preservas as mudanças realizadas nas colunas, bem como nas bases, optarei por transformá-las em um arquivo Excel (xlsx) para que estas mudanças perpetuem durante toda a análise e modelagem.
 

In [47]:
base_cadastral.to_excel('../data/processed/base_cadastral.xlsx', index=False)
base_info.to_excel('../data/processed/base_info.xlsx', index=False)
base_pag.to_excel('../data/processed/base_pagamentos_desenvolvimento.xlsx', index=False)